In [ ]:
# Ce script Python effectue une estimation de densité à noyau (KDE) sur des données
# de distribution d'espèces fournies par scikit-learn. Il tente ensuite de visualiser
# ces distributions sur une carte si les bibliothèques matplotlib et Basemap sont
# disponibles. Si matplotlib n'est pas installé, la partie de la visualisation est
# ignorée et un message est affiché à l'utilisateur.

import numpy as np
from sklearn.datasets import fetch_species_distributions
from sklearn.datasets._species_distributions import construct_grids
from sklearn.neighbors import KernelDensity

# Bloc try-except pour importer matplotlib et Basemap (qui est optionnel).
# Si une ImportError se produit (signifiant que la bibliothèque n'est pas installée),
# les variables basemap et plotting_available sont mises à False.
try:
    import matplotlib.pyplot as plt
    from mpl_toolkits.basemap import Basemap
    basemap = True
    plotting_available = True
except ImportError:
    basemap = False
    plotting_available = False

def load_data():
    """
    Charge les données de distribution des espèces à l'aide de la fonction
    fetch_species_distributions de scikit-learn.

    Returns:
        tuple: Un tuple contenant les données chargées et une liste des noms
               des deux espèces étudiées ('Bradypus Variegatus' et
               'Microryzomys Minutus').
    """
    data = fetch_species_distributions()
    species_names = ['Bradypus Variegatus', 'Microryzomys Minutus']
    return data, species_names

def prepare_training_data(data):
    """
    Prépare les données d'entraînement pour le modèle KDE. Extrait les latitudes
    et longitudes des données d'entraînement et crée un tableau NumPy où chaque
    ligne représente un point (latitude, longitude). Convertit également les
    coordonnées de degrés en radians, ce qui est nécessaire pour la métrique
    'haversine' utilisée dans KernelDensity. Crée également un tableau d'étiquettes
    indiquant l'espèce pour chaque point d'entraînement.

    Args:
        data (Bunch): Les données chargées par fetch_species_distributions.

    Returns:
        tuple: Un tuple contenant :
            - Xtrain (ndarray): Un tableau 2D des coordonnées d'entraînement
              (latitude, longitude) en radians.
            - ytrain (ndarray): Un tableau 1D d'entiers indiquant l'espèce
              correspondante pour chaque point dans Xtrain (0 pour la première
              espèce, 1 pour la seconde).
    """
    Xtrain = np.vstack([data['train']['dd lat'], data['train']['dd long']]).T
    ytrain = np.array([d.decode('ascii').startswith('micro')
                      for d in data['train']['species']], dtype='int')
    Xtrain *= np.pi / 180.  # Conversion des degrés en radians
    return Xtrain, ytrain

def prepare_grid(data):
    """
    Construit la grille de points sur laquelle la densité sera évaluée. Utilise
    la fonction construct_grids de scikit-learn pour créer les grilles de
    latitude et de longitude. Crée ensuite un maillage (meshgrid) à partir de
    ces grilles et applique un masque pour ne conserver que les points situés
    sur terre (en utilisant les données de couverture terrestre). Convertit
    également les coordonnées de la grille en radians.

    Args:
        data (Bunch): Les données chargées par fetch_species_distributions.

    Returns:
        tuple: Un tuple contenant :
            - X (ndarray): Un tableau 2D des longitudes de la grille.
            - Y (ndarray): Un tableau 2D des latitudes de la grille.
            - land_reference (ndarray): Une sous-section de la couverture terrestre
              utilisée comme référence.
            - land_mask (ndarray): Un masque booléen 1D indiquant les points de la
              grille situés sur terre.
            - xy (ndarray): Un tableau 2D des coordonnées (latitude, longitude)
              des points de la grille situés sur terre, en radians.
    """
    xgrid, ygrid = construct_grids(data)
    X, Y = np.meshgrid(xgrid[::5], ygrid[::5][::-1])  # Sous-échantillonnage de la grille
    land_reference = data.coverages[6][::5, ::5]
    land_mask = (land_reference > -9999).ravel()  # Masque pour les zones terrestres

    xy = np.vstack([Y.ravel(), X.ravel()]).T
    xy = xy[land_mask]
    xy *= np.pi / 180.  # Conversion des degrés en radians

    return X, Y, land_reference, land_mask, xy

def compute_kde(Xtrain, ytrain, species_index, bandwidth=0.04):
    """
    Calcule l'estimation de densité à noyau (KDE) pour une espèce spécifique.
    Utilise un noyau gaussien et la métrique de la distance haversine (adaptée
    aux données sphériques comme les coordonnées géographiques). Le modèle KDE
    est ajusté uniquement sur les points d'entraînement correspondant à l'espèce
    indiquée par species_index.

    Args:
        Xtrain (ndarray): Les données d'entraînement (latitude, longitude) en radians.
        ytrain (ndarray): Les étiquettes des espèces pour les données d'entraînement.
        species_index (int): L'indice de l'espèce pour laquelle calculer le KDE (0 ou 1).
        bandwidth (float, optional): La largeur de bande du noyau gaussien.
            Contrôle le lissage de l'estimation de densité. Defaults to 0.04.

    Returns:
        KernelDensity: Un objet KernelDensity ajusté aux données de l'espèce spécifiée.
    """
    kde = KernelDensity(bandwidth=bandwidth, metric='haversine',
                        kernel='tophat', algorithm='ball_tree')
    kde.fit(Xtrain[ytrain == species_index])
    return kde


def plot_species_distribution(X, Y, Z_list, land_reference, species_names):
    """
    Trace les distributions de densité estimées pour chaque espèce sur une seule
    figure, avec chaque distribution affichée sur un sous-graphique séparé.
    Utilise des contours remplis pour représenter la densité et ajoute une barre
    de couleur verticale pour indiquer les valeurs de densité. Si Basemap est
    disponible, une carte de base est ajoutée pour le contexte géographique.
    Sinon, les contours de la référence terrestre sont affichés.

    Args:
        X (ndarray): Les longitudes de la grille.
        Y (ndarray): Les latitudes de la grille.
        Z_list (list): Une liste de tableaux 2D représentant les densités estimées
                       pour chaque espèce.
        land_reference (ndarray): La référence de la couverture terrestre.
        species_names (list): Une liste des noms des espèces.
    """
    if not plotting_available:
        print("Skipping plot: matplotlib not available.")
        return

    # Ajuster la taille de la figure pour une meilleure lisibilité
    plt.figure(figsize=(16, 8))  # Taille augmentée pour afficher les deux cartes côte à côte

    for i, (Z, species_name) in enumerate(zip(Z_list, species_names)):
        plt.subplot(1, 2, i + 1)

        # Définir les niveaux pour les contours
        levels = np.linspace(0, Z.max(), 25)
        contour = plt.contourf(X, Y, Z, levels=levels, cmap='Reds', alpha=0.8)

        # Ajouter une carte de base si disponible
        if basemap:
            m = Basemap(projection='cyl', llcrnrlat=Y.min(), urcrnrlat=Y.max(),
                        llcrnrlon=X.min(), urcrnrlon=X.max(), resolution='c')
            m.drawcoastlines()
            m.drawcountries()
        else:
            plt.contour(X, Y, land_reference, levels=[-9999], colors="k", linestyles="solid")
            plt.xticks([])
            plt.yticks([])

        # Ajouter une barre de couleur verticale pour les densités
        cbar = plt.colorbar(contour, orientation='vertical', pad=0.02)
        cbar.set_label('Densité estimée (points/km²)')

        # Ajouter des titres, des étiquettes d'axes et des graduations
        plt.title(f"Distribution de {species_name}")
        plt.xlabel("Longitude (°)")
        plt.ylabel("Latitude (°)")
        plt.xticks(np.arange(-90, -29, 10))  # Graduations pour la longitude
        plt.yticks(np.arange(-60, 16, 15))  # Graduations pour la latitude

    plt.tight_layout()
    plt.show()


def visualize_dataset(Xtrain, ytrain, species_names):
    """
    Visualise les points de données d'entraînement pour chaque espèce sur des
    sous-graphiques séparés. Chaque point est coloré en fonction de l'espèce.
    Si Basemap est disponible, une carte de base de l'Amérique du Sud est
    affichée en arrière-plan. Sinon, les limites de longitude et de latitude
    pour l'Amérique du Sud sont définies et une grille est ajoutée.

    Args:
        Xtrain (ndarray): Les données d'entraînement (latitude, longitude) en radians.
        ytrain (ndarray): Les étiquettes des espèces pour les données d'entraînement.
        species_names (list): Une liste des noms des espèces.
    """
    if not plotting_available:
        print("Skipping raw dataset visualization: matplotlib not available.")
        return

    # Ajuster la taille de la figure pour une meilleure lisibilité
    plt.figure(figsize=(16, 8))  # Taille augmentée pour améliorer la visibilité

    for i, species_name in enumerate(species_names):
        plt.subplot(1, 2, i + 1)

        # Scatter plot des points d'entraînement
        plt.scatter(Xtrain[ytrain == i, 1] * 180 / np.pi,  # longitude
                    Xtrain[ytrain == i, 0] * 180 / np.pi,  # latitude
                    label=f"Présence de {species_name}", alpha=0.6, c='red')

        # Ajouter une carte de base si disponible
        if basemap:
            m = Basemap(projection='cyl', llcrnrlat=-60, urcrnrlat=15,  # Limites pour l'Amérique du Sud
                        llcrnrlon=-90, urcrnrlon=-30, resolution='c')
            m.drawcoastlines()
            m.drawcountries()
        else:
            plt.xlim(-90, -30)  # Limites de longitude pour l'Amérique du Sud
            plt.ylim(-60, 15)   # Limites de latitude pour l'Amérique du Sud
            plt.grid(True, linestyle='--', alpha=0.5)

        # Ajouter des légendes, des titres et des graduations
        plt.title(f"Visualisation des données : {species_name}")
        plt.xlabel("Longitude (°)")
        plt.ylabel("Latitude (°)")
        plt.xticks(np.arange(-90, -29, 10))  # Graduations pour la longitude
        plt.yticks(np.arange(-60, 16, 15))  # Graduations pour la latitude
        plt.legend(loc="upper right")

    plt.tight_layout()
    plt.show()


def main():
    """
    Fonction principale qui orchestre le chargement des données, la préparation
    des données d'entraînement et de la grille, le calcul des estimations de
    densité à noyau pour chaque espèce et la visualisation des résultats.
    """
    data, species_names = load_data()
    Xtrain, ytrain = prepare_training_data(data)
    X, Y, land_reference, land_mask, xy = prepare_grid(data)

    print(f"Xtrain shape: {Xtrain.shape}, ytrain shape: {ytrain.shape}")
    print(f"Grid shape: X={X.shape}, Y={Y.shape}, land_mask={land_mask.shape}")

    visualize_dataset(Xtrain, ytrain, species_names)

    if plotting_available:
        Z_list = []
        for i in range(2):
            print(f" - computing KDE for {species_names[i]}")
            kde = compute_kde(Xtrain, ytrain, species_index=i)

            # Préparer Z (les estimations de densité sur la grille terrestre)
            Z = -9999 + np.zeros(land_mask.shape[0])
            Z[land_mask] = np.exp(kde.score_samples(xy))
            print(f"Z shape before reshape: {Z.shape}")
            Z = Z.reshape(X.shape)  # Remodeler pour correspondre à la grille
            print(f"Z shape after reshape: {Z.shape}")

            Z_list.append(Z)

        print(" - plotting distributions")
        plot_species_distribution(X, Y, Z_list, land_reference, species_names)

if __name__ == "__main__":
    main()

In [ ]:
import numpy as np
import tkinter as tk
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_species_distributions
from sklearn.datasets._species_distributions import construct_grids
from sklearn.neighbors import KernelDensity

# Chargement des données
data, species_names = fetch_species_distributions(), ['Bradypus Variegatus', 'Microryzomys Minutus']

# Préparation des données
def prepare_training_data(data):
    Xtrain = np.vstack([data['train']['dd lat'], data['train']['dd long']]).T
    ytrain = np.array([d.decode('ascii').startswith('micro') for d in data['train']['species']], dtype='int')
    Xtrain *= np.pi / 180.
    return Xtrain, ytrain

def prepare_grid(data):
    xgrid, ygrid = construct_grids(data)
    X, Y = np.meshgrid(xgrid[::5], ygrid[::5][::-1])
    land_reference = data.coverages[6][::5, ::5]
    land_mask = (land_reference > -9999).ravel()

    xy = np.vstack([Y.ravel(), X.ravel()]).T
    xy = xy[land_mask]
    xy *= np.pi / 180.

    return X, Y, land_reference, land_mask, xy

Xtrain, ytrain = prepare_training_data(data)
X, Y, land_reference, land_mask, xy = prepare_grid(data)

# Application principale Tkinter
class KDEVisualizerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Visualisation KDE sur données d'espèces")

        # Paramètres initiaux
        self.bandwidth = tk.DoubleVar(value=0.04)
        self.kernel = tk.StringVar(value="gaussian")
        self.show_presence = tk.BooleanVar(value=True)

        # Interface utilisateur
        self.setup_widgets()

    def setup_widgets(self):
        controls_frame = tk.Frame(self.root)
        controls_frame.pack(side=tk.TOP, fill=tk.X)

        tk.Label(controls_frame, text="Bande passante :").pack(side=tk.LEFT)
        self.bandwidth_entry = tk.Entry(controls_frame, textvariable=self.bandwidth, width=6)
        self.bandwidth_entry.pack(side=tk.LEFT, padx=5)

        tk.Label(controls_frame, text="Noyau :").pack(side=tk.LEFT)
        self.kernel_menu = ttk.Combobox(controls_frame, textvariable=self.kernel, 
                                        values=["gaussian", "tophat", "epanechnikov"], width=12)
        self.kernel_menu.pack(side=tk.LEFT, padx=5)

        self.presence_check = tk.Checkbutton(controls_frame, text="Afficher données de présence", variable=self.show_presence)
        self.presence_check.pack(side=tk.LEFT, padx=10)

        self.plot_button = tk.Button(controls_frame, text="Tracer", command=self.plot_distributions)
        self.plot_button.pack(side=tk.LEFT, padx=10)

        self.figure = plt.Figure(figsize=(12, 6))
        self.canvas = FigureCanvasTkAgg(self.figure, master=self.root)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

    def plot_distributions(self):
        self.figure.clf()

        bandwidth = self.bandwidth.get()
        kernel = self.kernel.get()

        ax1 = self.figure.add_subplot(1, 2, 1)
        ax2 = self.figure.add_subplot(1, 2, 2)

        Z_list = []
        for i in range(2):
            kde = KernelDensity(bandwidth=bandwidth, metric='haversine',
                                kernel=kernel, algorithm='ball_tree')
            kde.fit(Xtrain[ytrain == i])

            log_dens = kde.score_samples(xy)
            Z = np.full(land_mask.shape, -9999.)
            Z[land_mask] = np.exp(log_dens)
            Z = Z.reshape(X.shape)

            Z_list.append(Z)

        for idx, (ax, Z, name) in enumerate(zip([ax1, ax2], Z_list, species_names)):
            levels = np.linspace(0, Z.max(), 25)
            contour = ax.contourf(X, Y, Z, levels=levels, cmap='Reds', alpha=0.8)
            #self.figure.colorbar(contour, ax=ax)
            # Ajouter une barre de couleur verticale pour les densités
            cbar = plt.colorbar(contour, orientation='vertical', pad=0.02)
            cbar.set_label('Densité estimée (points/km²)')  # Ajout du label

            # Ajouter les "contours" du masque terrestre
            ax.contour(X, Y, land_reference, levels=[-9998], colors='black',linestyles='solid',linewidths=0.5)

            if self.show_presence.get():
                ax.scatter(Xtrain[ytrain == idx, 1] * 180/np.pi,
                           Xtrain[ytrain == idx, 0] * 180/np.pi,
                           c='blue', s=10, alpha=0.5, label='Présences réelles')

            ax.set_title(f"Distribution : {name}")
            ax.set_xlabel("Longitude (en degrés)")
            ax.set_ylabel("Latitude (en degrés)")
            ax.legend()

        self.figure.tight_layout()
        self.canvas.draw()

# Lancement
if __name__ == "__main__":
    root = tk.Tk()
    app = KDEVisualizerApp(root)
    root.mainloop()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
